In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20200713.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,7/9/2020 21:31:14,But I'm a Cheerleader!,Happy Together,"The Adventures of Priscilla, Queen of the Desert",Hairspray,The Living End,"To Wong Foo, Thanks for Everything! Julie Newmar",LOTS OF SLIME,No Theme/Free-for-all,LADIES NIGHT,THIS SOUNDTRACK BANGS,"TECHNICALLY, FOR CHILDREN",TOXIC MASCULINITY A GO-GO!,ENNIO MORRICONE REST IN PEACE
1,7/9/2020 21:32:30,But I'm a Cheerleader!,"To Wong Foo, Thanks for Everything! Julie Newmar",Happy Together,The Living End,"The Adventures of Priscilla, Queen of the Desert",Hairspray,LOTS OF SLIME,THIS SOUNDTRACK BANGS,"TECHNICALLY, FOR CHILDREN",ENNIO MORRICONE REST IN PEACE,TOXIC MASCULINITY A GO-GO!,LADIES NIGHT,No Theme/Free-for-all
2,7/9/2020 21:47:29,Happy Together,The Living End,Hairspray,But I'm a Cheerleader!,"The Adventures of Priscilla, Queen of the Desert","To Wong Foo, Thanks for Everything! Julie Newmar",No Theme/Free-for-all,LADIES NIGHT,"TECHNICALLY, FOR CHILDREN",THIS SOUNDTRACK BANGS,ENNIO MORRICONE REST IN PEACE,LOTS OF SLIME,TOXIC MASCULINITY A GO-GO!
3,7/9/2020 21:52:29,But I'm a Cheerleader!,"The Adventures of Priscilla, Queen of the Desert",Hairspray,"To Wong Foo, Thanks for Everything! Julie Newmar",Happy Together,The Living End,"TECHNICALLY, FOR CHILDREN",LADIES NIGHT,No Theme/Free-for-all,NaN,NaN,NaN,NaN
4,7/9/2020 22:01:00,The Living End,But I'm a Cheerleader!,"The Adventures of Priscilla, Queen of the Desert",Hairspray,Happy Together,"To Wong Foo, Thanks for Everything! Julie Newmar",TOXIC MASCULINITY A GO-GO!,LADIES NIGHT,THIS SOUNDTRACK BANGS,LOTS OF SLIME,"TECHNICALLY, FOR CHILDREN",ENNIO MORRICONE REST IN PEACE,No Theme/Free-for-all
5,7/9/2020 22:05:06,"The Adventures of Priscilla, Queen of the Desert","To Wong Foo, Thanks for Everything! Julie Newmar",Happy Together,The Living End,But I'm a Cheerleader!,Hairspray,"TECHNICALLY, FOR CHILDREN",LADIES NIGHT,LOTS OF SLIME,THIS SOUNDTRACK BANGS,NaN,NaN,NaN
6,7/9/2020 22:33:30,Happy Together,"To Wong Foo, Thanks for Everything! Julie Newmar","The Adventures of Priscilla, Queen of the Desert",But I'm a Cheerleader!,The Living End,Hairspray,THIS SOUNDTRACK BANGS,ENNIO MORRICONE REST IN PEACE,TOXIC MASCULINITY A GO-GO!,"TECHNICALLY, FOR CHILDREN",LADIES NIGHT,LOTS OF SLIME,No Theme/Free-for-all
7,7/10/2020 14:27:51,"The Adventures of Priscilla, Queen of the Desert","To Wong Foo, Thanks for Everything! Julie Newmar",But I'm a Cheerleader!,The Living End,Happy Together,Hairspray,ENNIO MORRICONE REST IN PEACE,LOTS OF SLIME,LADIES NIGHT,TOXIC MASCULINITY A GO-GO!,THIS SOUNDTRACK BANGS,"TECHNICALLY, FOR CHILDREN",No Theme/Free-for-all
8,7/10/2020 16:08:37,The Living End,But I'm a Cheerleader!,"The Adventures of Priscilla, Queen of the Desert",Hairspray,"To Wong Foo, Thanks for Everything! Julie Newmar",Happy Together,ENNIO MORRICONE REST IN PEACE,TOXIC MASCULINITY A GO-GO!,LOTS OF SLIME,"TECHNICALLY, FOR CHILDREN",LADIES NIGHT,THIS SOUNDTRACK BANGS,NaN
9,7/10/2020 17:13:08,The Living End,But I'm a Cheerleader!,Hairspray,"The Adventures of Priscilla, Queen of the Desert","To Wong Foo, Thanks for Everything! Julie Newmar",Happy Together,"TECHNICALLY, FOR CHILDREN",ENNIO MORRICO

In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(["But I'm a Cheerleader!", 'Happy Together',
       'The Adventures of Priscilla, Queen of the Desert', 'Hairspray',
       'The Living End',
       'To Wong Foo, Thanks for Everything! Julie Newmar'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{"But I'm a Cheerleader!": <Candidate('But I'm a Cheerleader!')>,
 'Happy Together': <Candidate('Happy Together')>,
 'The Adventures of Priscilla, Queen of the Desert': <Candidate('The Adventures of Priscilla, Queen of the Desert')>,
 'Hairspray': <Candidate('Hairspray')>,
 'The Living End': <Candidate('The Living End')>,
 'To Wong Foo, Thanks for Everything! Julie Newmar': <Candidate('To Wong Foo, Thanks for Everything! Julie Newmar')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(But I'm a Cheerleader!, Happy Together, The Adventures of Priscilla, Queen of the Desert, Hairspray, The Living End, To Wong Foo, Thanks for Everything! Julie Newmar)>,
 <Ballot(But I'm a Cheerleader!, To Wong Foo, Thanks for Everything! Julie Newmar, Happy Together, The Living End, The Adventures of Priscilla, Queen of the Desert, Hairspray)>,
 <Ballot(Happy Together, The Living End, Hairspray, But I'm a Cheerleader!, The Adventures of Priscilla, Queen of the Desert, To Wong Foo, Thanks for Everything! Julie Newmar)>,
 <Ballot(But I'm a Cheerleader!, The Adventures of Priscilla, Queen of the Desert, Hairspray, To Wong Foo, Thanks for Everything! Julie Newmar, Happy Together, The Living End)>,
 <Ballot(The Living End, But I'm a Cheerleader!, The Adventures of Priscilla, Queen of the Desert, Hairspray, Happy Together, To Wong Foo, Thanks for Everything! Julie Newmar)>,
 <Ballot(The Adventures of Priscilla, Queen of the Desert, To Wong Foo, Thanks for Everything! Julie Newmar, H

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                                           Votes  Status
------------------------------------------------  -------  --------
But I'm a Cheerleader!                                  3  Hopeful
The Living End                                          3  Hopeful
The Adventures of Priscilla, Queen of the Desert        2  Hopeful
Happy Together                                          2  Rejected
To Wong Foo, Thanks for Everything! Julie Newmar        0  Rejected
Hairspray                                               0  Rejected

ROUND 2
Candidate                                           Votes  Status
------------------------------------------------  -------  --------
The Living End                                          4  Hopeful
But I'm a Cheerleader!                                  3  Hopeful
The Adventures of Priscilla, Queen of the Desert        3  Rejected
Happy Together                                          0  Rejected
To Wong Foo, Thanks for Everything! Juli

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                        Votes  Status
-----------------------------  -------  --------
TECHNICALLY, FOR CHILDREN            3  Hopeful
ENNIO MORRICONE REST IN PEACE        2  Hopeful
LOTS OF SLIME                        2  Hopeful
THIS SOUNDTRACK BANGS                1  Hopeful
TOXIC MASCULINITY A GO-GO!           1  Hopeful
No Theme/Free-for-all                1  Rejected
LADIES NIGHT                         0  Rejected

ROUND 2
Candidate                        Votes  Status
-----------------------------  -------  --------
TECHNICALLY, FOR CHILDREN            4  Hopeful
LOTS OF SLIME                        2  Hopeful
ENNIO MORRICONE REST IN PEACE        2  Hopeful
THIS SOUNDTRACK BANGS                1  Rejected
TOXIC MASCULINITY A GO-GO!           1  Rejected
No Theme/Free-for-all                0  Rejected
LADIES NIGHT                         0  Rejected

ROUND 3
Candidate                        Votes  Status
-----------------------------  -------  --------
TECHNICA